In [1]:
%load_ext autoreload
%autoreload 2
from utils import *

c:\virtual Environments\transformer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### The original (German, English) dataset download links do not work - replace download links

In [2]:
from torchtext.datasets import multi30k

multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"
multi30k.URL["test"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz"

multi30k.MD5["train"] = "20140d013d05dd9a72dfde46478663ba05737ce983f478f960c1123c6671be5e"
multi30k.MD5["valid"] = "a7aa20e9ebd5ba5adce7909498b94410996040857154dab029851af3a866da8c"
multi30k.MD5["test"] = "6d1ca1dba99e2c5dd54cae1226ff11c2551e6ce63527ebb072a1f70f72a5cd36"

# Load HuggingFace Tokenizers

In [3]:
# These (tokenizers and the model) all have global scope
from transformers import  AutoTokenizer,GPT2Tokenizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#
tknzr_de = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")
tknzr_de.add_special_tokens({'pad_token': '[PAD]', 'bos_token': '[BOS]', 'eos_token' : '[EOS]', 'unk_token':'[UNK]'})
#
tknzr_en = GPT2Tokenizer.from_pretrained('gpt2')
tknzr_en.add_special_tokens({'pad_token': '[PAD]', 'bos_token': '[BOS]', 'eos_token' : '[EOS]', 'unk_token':'[UNK]'})
#
padSrc = tknzr_de(tknzr_de.pad_token)['input_ids'][0]
padTgt = tknzr_en(tknzr_en.pad_token)['input_ids'][0]
#
lenSrc = len(tknzr_de)
lenTgt = len(tknzr_en)
#---------
pad_IdDe = torch.tensor(tknzr_de(tknzr_de.pad_token)['input_ids'][0],device = device)
bs_idDe = torch.tensor(tknzr_de(tknzr_de.bos_token)['input_ids'][0], device = device)
eos_idDe = torch.tensor(tknzr_de(tknzr_de.eos_token)['input_ids'][0], device = device)
#
pad_IdEn = torch.tensor(tknzr_en(tknzr_en.pad_token)['input_ids'][0], device = device)
bs_idEn = torch.tensor(tknzr_en(tknzr_en.bos_token)['input_ids'][0], device = device)
eos_idEn = torch.tensor(tknzr_en(tknzr_en.eos_token)['input_ids'][0], device = device)

## Load Transformer model trained with HuggingFace tokenization

In [4]:
modelHF = make_model(lenSrc, lenTgt, N=6, HF = 1)
print("Loading trained Huggingface model")
modelHF.load_state_dict(
    torch.load("multi30k_model_final_HF.pt", map_location=torch.device("cpu"))
)

Loading trained Huggingface model


RuntimeError: Error(s) in loading state_dict for EncoderDecoder:
	Missing key(s) in state_dict: "encoder.layers.0.norm1.a_2", "encoder.layers.0.norm1.b_2", "encoder.layers.0.norm2.a_2", "encoder.layers.0.norm2.b_2", "encoder.layers.1.norm1.a_2", "encoder.layers.1.norm1.b_2", "encoder.layers.1.norm2.a_2", "encoder.layers.1.norm2.b_2", "encoder.layers.2.norm1.a_2", "encoder.layers.2.norm1.b_2", "encoder.layers.2.norm2.a_2", "encoder.layers.2.norm2.b_2", "encoder.layers.3.norm1.a_2", "encoder.layers.3.norm1.b_2", "encoder.layers.3.norm2.a_2", "encoder.layers.3.norm2.b_2", "encoder.layers.4.norm1.a_2", "encoder.layers.4.norm1.b_2", "encoder.layers.4.norm2.a_2", "encoder.layers.4.norm2.b_2", "encoder.layers.5.norm1.a_2", "encoder.layers.5.norm1.b_2", "encoder.layers.5.norm2.a_2", "encoder.layers.5.norm2.b_2". 
	Unexpected key(s) in state_dict: "encoder.layers.0.sublayer.0.norm.a_2", "encoder.layers.0.sublayer.0.norm.b_2", "encoder.layers.0.sublayer.1.norm.a_2", "encoder.layers.0.sublayer.1.

# Load Spacy tokenizers and vocabulary

In [11]:
RUN_EXAMPLES = True
def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)

def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)

# global variables used later in the script
spacy_de, spacy_en = show_example(load_tokenizers)
vocab_src, vocab_tgt = show_example(load_vocab, args=[spacy_de, spacy_en])

Finished.
Vocabulary sizes:
8316
6384


In [13]:
def check_outputs(
    valid_dataloader,
    model,
    vocab_src,
    vocab_tgt, #
    n_examples=15,
    pad_idx=2,
    eos_string="</s>", 
):
    results = [()] * n_examples
    resultsHF = [()]*n_examples
    padSrcHF = tknzr_de(tknzr_de.pad_token)['input_ids'][0]
    padTgtHF = tknzr_en(tknzr_en.pad_token)['input_ids'][0]
    for idx in range(n_examples):
        print("\nExample %d ========\n" % idx)
        b = next(iter(valid_dataloader))
        ###
        # SPACY
        ###
        rb = Batch(b[0], b[1], pad_idx) # change the collate function for valid_dataloader to return b[0],b[1],b[2],b[3]
                                        # b[2],b[3] are the source and target tok2Id for HF model

        src_tokens = [
            vocab_src.get_itos()[x] for x in rb.src[0] if x != pad_idx
        ]
        tgt_tokens = [
            vocab_tgt.get_itos()[x] for x in rb.tgt[0] if x != pad_idx
        ]

        print(
            "Source Spacy Text (Input)        : "
            + " ".join(src_tokens).replace("\n", "")
        )
        print(
            "Target Spacy Text (Ground Truth) : "
            + " ".join(tgt_tokens).replace("\n", "")
        )
        model_out = greedy_decode(model, rb.src, rb.src_mask, 72, 0)[0]
        model_txt = (
            " ".join(
                [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
            ).split(eos_string, 1)[0]
            + eos_string
        )
        print("Model Spacy Output               : " + model_txt.replace("\n", ""))
        results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
        print("========\n")
        ###
        # HUGGING FACE
        ###
        rb = BatchHF(b[2], b[3], padSrcHF, padTgtHF)
        src_tokens = [tknzr_de.decode(x) for x in rb.src[0] if x!= padSrcHF]
        tgt_tokens = [tknzr_en.decode(x) for x in rb.tgt[0] if x!= padTgtHF]
        print(
            "Source HF Text  (Input)        : "
            + " ".join(src_tokens).replace("\n", "")
        )
        print(
            "Target HF Text  (Ground Truth) : "
            + " ".join(tgt_tokens).replace("\n", "")
        )
        model_out = greedy_decode(modelHF, rb.src, rb.src_mask, 72, start_symbol = tknzr_en(tknzr_en.bos_token)['input_ids'][0])[0]
        model_txt = (
            " ".join(
                [tknzr_en.decode(x) for x in model_out if x != padTgtHF]
            ).split(tknzr_en.eos_token, 1)[0]
            + tknzr_en.eos_token
        )
        print("Model HF Output: " + model_txt.replace("\n", ""))

        resultsHF[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
    return results#,resultsHF


def run_model_example(n_examples=25):
    global vocab_src, vocab_tgt, spacy_de, spacy_en

    print("Preparing Data ...")
    valid_dataloader = create_dataloaders(
        torch.device("cpu"),
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=1,
        is_distributed=False,
        tknzr_de=tknzr_de,
        tknzr_en=tknzr_en
    )

    print("Loading Trained Spacy Model ...")
    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(
        torch.load("multi30k_model_final_Spacy.pt", map_location=torch.device("cpu"))
    )
    print("Comparing Model Outputs:")
    example_data = check_outputs(
        valid_dataloader, model, vocab_src, vocab_tgt, n_examples=n_examples
    )
    return model, example_data

execute_example(run_model_example)

Preparing Data ...
Loading Trained Spacy Model ...
Comparing Model Outputs:

Example 0 ========

Source Spacy Text (Input)        : <s> Ein Mann mit einem Namensschild sitzt in einem Stuhl . </s>
Target Spacy Text (Ground Truth) : <s> A man with a name tag on is sitting in a chair . </s>
Attention Matrix = torch.Size([1, 8, 72, 72])
torch.Size([1, 1, 1, 72])
tensor([[ 3.6139,  3.1037,  9.0814,  ...,  2.1154,  1.4405,  0.6778],
        [ 3.8724,  2.6457,  8.4249,  ...,  4.6558,  3.7353,  4.1025],
        [ 3.7073,  1.7089,  9.6498,  ...,  1.1027,  1.1904,  1.1479],
        ...,
        [ 0.6274, -0.6002,  1.4411,  ...,  3.9147,  3.5533,  4.2915],
        [ 0.3844, -0.8139,  0.9192,  ...,  3.0393,  3.4961,  4.0271],
        [ 0.9882, -0.2461,  1.0973,  ...,  3.2897,  3.5954,  4.6826]],
       grad_fn=<SliceBackward0>)
tensor([[ 3.6139e+00,  3.1037e+00,  9.0814e+00,  ..., -1.0000e+09,
         -1.0000e+09, -1.0000e+09],
        [ 3.8724e+00,  2.6457e+00,  8.4249e+00,  ..., -1.0000e+09,
  

NameError: name 'modelHF' is not defined